In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import keras

In [10]:
tf.__version__, keras.__version__

('2.16.1', '3.3.3')

In [11]:
# This is required to download Gemma model
# import kagglehub
# kagglehub.login()

# Build model

In [12]:
from keras_llm_light.models import gemma

LORA_RANK = 4
LORA_APLHA = 32.0
MAX_SEQ_LENGTH = 512

optimizer = keras.optimizers.Adam(learning_rate=0.0001)

trainer, llm = gemma.build_gemma_llm(
    lora_rank=LORA_RANK,
    lora_alpha=LORA_APLHA,
    max_sequence_len=MAX_SEQ_LENGTH,
    optimizer=optimizer,
)

Building Gemma 2B model on CPU ...


normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

...

 Non-trainable params: 0 (0.00 B)

2024-06-06 18:26:49.557361: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


Single block trainable weights:
- decoder_block/attention/query/lora_A/kernel
- decoder_block/attention/query/lora_B/kernel
- decoder_block/attention/value/lora_A/kernel
- decoder_block/attention/value/lora_B/kernel


Preparing/Quantizing weights: 100%|██████████| 18/18 [00:36<00:00,  2.02s/it]

Single block memory usage:
 - decoder_block/pre_attention_norm/scale                       =>  0.004 MB  float16
 - decoder_block/attention/query/kernel                         =>  8.000 MB  float16
 - decoder_block/attention/key/kernel                           =>  1.000 MB  float16
 - decoder_block/attention/value/kernel                         =>  1.000 MB  float16
 - decoder_block/attention/attention_output/kernel              =>  8.000 MB  float16
 - decoder_block/attention/query/lora_A/kernel                  =>  0.031 MB  float32
 - decoder_block/attention/query/lora_B/kernel                  =>  0.031 MB  float32
 - decoder_block/attention/value/lora_A/kernel                  =>  0.031 MB  float32
 - decoder_block/attention/value/lora_B/kernel                  =>  0.004 MB  float32
 - decoder_block/pre_ffw_norm/scale                             =>  0.002 MB  uint8
 - decoder_block/ffw_gating/kernel                              => 33.130 MB  uint8
 - decoder_block/ffw_gating_2/k

# Train model

In [14]:
ds = tfds.load("imdb_reviews", split="train")

for document in ds:
    break

document

2024-06-06 18:29:52.070280: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


{'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 'text': <tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">}

In [15]:
SEPARATOR  = "Sentiment: "
LABEL_TO_SENTIMENT = tf.constant(["Negative", "Positive"])

def prepare_text(document):
    text = document['text']
    sentiment = LABEL_TO_SENTIMENT[document['label']]
    return text + f"\n{SEPARATOR}" + sentiment

In [16]:
BATCH_SIZE = 1

train_ds = (
    ds.map(prepare_text)
    .repeat(-1)
    .shuffle(2000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [17]:
train_ds_processed = train_ds.map(llm.preprocessor)
train_ds_processed_iter = iter(train_ds_processed)

In [27]:
for sample in train_ds:
    break

parts = sample[0].numpy().decode("utf-8").split(SEPARATOR)
query = parts[0] + SEPARATOR
print(query)
print(parts[1])

An awful film! It must have been up against some real stinkers to be nominated for the Golden Globe. They've taken the story of the first famous female Renaissance painter and mangled it beyond recognition. My complaint is not that they've taken liberties with the facts; if the story were good, that would perfectly fine. But it's simply bizarre -- by all accounts the true story of this artist would have made for a far better film, so why did they come up with this dishwater-dull script? I suppose there weren't enough naked people in the factual version. It's hurriedly capped off in the end with a summary of the artist's life -- we could have saved ourselves a couple of hours if they'd favored the rest of the film with same brevity.
Sentiment: 
Negative


2024-06-06 18:32:34.023783: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [31]:
response_text = llm.generate(query, max_length=256)
print(query + response_text.numpy().decode("utf-8"))

Generating text: 255it [00:13, 19.30it/s]              

An awful film! It must have been up against some real stinkers to be nominated for the Golden Globe. They've taken the story of the first famous female Renaissance painter and mangled it beyond recognition. My complaint is not that they've taken liberties with the facts; if the story were good, that would perfectly fine. But it's simply bizarre -- by all accounts the true story of this artist would have made for a far better film, so why did they come up with this dishwater-dull script? I suppose there weren't enough naked people in the factual version. It's hurriedly capped off in the end with a summary of the artist's life -- we could have saved ourselves a couple of hours if they'd favored the rest of the film with same brevity.
Sentiment: 1/10
Acting: 1/10
Direction: 1/10
Story: 1/10
Overall: 1/10
I'm not sure what I expected from this movie. I've never heard of this artist before, and I'm not sure if I've heard of her before. I'm not sure if I've ever heard of this movie


In [32]:
metrics = trainer.fit(llm, train_ds_processed, epochs=1, steps_per_epoch=1000)

0it [00:00, ?it/s]

Tracing block_forward


I0000 00:00:1717691744.699085  169936 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_7', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1717691745.226579  169939 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_10', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1717691745.982619  169942 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 24 bytes spill stores, 24 bytes spill loads

2024-06-06 18:35:48.336950: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1016.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-06-06 18:35:48.339210: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:580 : 

ResourceExhaustedError: Out of memory while trying to allocate 1065353216 bytes. [Op:__inference_partial_train_loss_step_90832]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
metrics_df = pd.DataFrame(metrics)
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(metrics_df["loss"], label="loss")
plt.subplot(122) 
plt.plot(metrics_df["accuracy"], label="accuracy")

In [ ]:
for sample in train_ds:
    break

sample

In [ ]:
split_key = "\nAnswer: "
parts = sample[0].numpy().decode("utf-8").split(split_key)
query = parts[0] + split_key
print(query)
print(parts[1])

In [ ]:
response_text = model.generate("Is is Żywiec Jasne or JAsne Pelne?", max_length=128)
print(response_text.numpy().decode("utf-8"))